In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, ParameterGrid, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, f1_score, pairwise_distances
import sklearn.metrics as metrics
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.spatial.distance import cdist, mahalanobis
from scipy.spatial import distance
from scipy.stats import mode
from scipy.stats import chi2

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from joblib import parallel_backend
import joblib

import threading
import os

In [12]:
print(sklearn.__version__)

1.5.1


In [13]:
thread_count = threading.active_count()
print('Number of Threads:', thread_count)

num_threads = os.cpu_count()
print(f'Number of logical cores: {num_threads}')

Number of Threads: 6
Number of logical cores: 8


In [14]:
df = pd.read_csv('df_modelling.csv', index_col=0)

In [15]:
df.head()

,Area,AspectRation,Solidity,roundness,ShapeFactor1,ShapeFactor4,ShapeFactor5,Sort order,Extent,ShapeFactor6,Constantness,Colour_black,Colour_brown,Colour_green,Colour_white,Class
0,84648,1.767000,0.975443,0.830027,0.005197,3.487008,0.984066,0.134791,0.767184,50.809833,1,False,True,False,False,CALI
1,39704,1.456767,0.989977,0.918880,0.006842,1.242335,0.997891,0.898848,0.757739,148.508874,1,True,False,False,False,DERMASON
2,35835,1.236904,0.990191,0.951588,0.006633,2.813489,0.998803,0.539684,0.790885,82.107117,1,False,False,True,False,SEKER
3,223035,1.621004,0.987365,0.848673,0.003052,3.074472,0.993144,0.149130,0.761092,147.347735,1,False,True,False,False,BOMBAY
4,41957,1.546802,0.991657,0.895904,0.006863,2.213954,0.996497,0.447635,0.721543,22.893826,1,False,True,False,False,SIRA


In [16]:
classes = list(df['Class'].value_counts().index)
classes

['DERMASON', 'SIRA', 'SEKER', 'HOROZ', 'CALI', 'BARBUNYA', 'BOMBAY']

In [17]:
X = np.float_(df.drop('Class', axis=1).values)
y = np.array([classes.index(x) for x in df['Class']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
_dt = DecisionTreeClassifier(random_state=42)
_dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [19]:
ccp_alphas = _dt.cost_complexity_pruning_path(X_train, y_train)['ccp_alphas']

In [20]:
len(ccp_alphas)

101

In [21]:
_dt.get_depth()

19

In [22]:
tree_grid = {
    'handling_unbalance_data': ['smote', 'under', 'imbalanced', 'none'],
    'criterion': ['gini', 'entropy'],
    'ccp_alpha': ccp_alphas,
    'max_depth': [None, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 8]
}

In [23]:
class CustomDecisionTree(BaseEstimator, ClassifierMixin):
    def __init__(
            self, 
            handling_unbalance_data='none', 
            criterion='gini', ccp_alpha=0.0, 
            max_depth=None, 
            min_samples_split=2, 
            min_samples_leaf=1):
        self.handling_unbalance_data = handling_unbalance_data
        self.criterion = criterion
        self.ccp_alpha = ccp_alpha
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        if self.handling_unbalance_data == 'imbalanced':
            self.clf = DecisionTreeClassifier(
                class_weight='balanced',
                criterion=self.criterion, 
                ccp_alpha=self.ccp_alpha, 
                max_depth=self.max_depth, 
                min_samples_split=self.min_samples_split, 
                min_samples_leaf=self.min_samples_leaf,
                random_state=42)
        else:
            self.clf = DecisionTreeClassifier(
                criterion=self.criterion, 
                ccp_alpha=self.ccp_alpha, 
                max_depth=self.max_depth, 
                min_samples_split=self.min_samples_split, 
                min_samples_leaf=self.min_samples_leaf,
                random_state=42)
        
    def fit(self, X, y):
        if self.handling_unbalance_data == 'smote':
            sm = SMOTE(random_state=42)
            X, y = sm.fit_resample(X, y)
        elif self.handling_unbalance_data == 'under':
            rus = RandomUnderSampler(random_state=42)
            X, y =  rus.fit_resample(X, y)
        self.clf.fit(X, y)
        self.classes_ = self.clf.classes_
        self.n_classes_ = self.clf.n_classes_
        self.n_features_in_ = self.clf.n_features_in_
        if hasattr(self.clf, 'feature_names_in_'):
            self.feature_names_in_ = self.clf.feature_names_in_
        return self
    
    def predict(self, X):
        return self.clf.predict(X)

    def get_params(self, deep=True):
        return {"handling_unbalance_data": self.handling_unbalance_data, 
                "criterion": self.criterion, 
                "ccp_alpha": self.ccp_alpha, 
                "max_depth": self.max_depth, 
                "min_samples_split": self.min_samples_split, 
                "min_samples_leaf": self.min_samples_leaf}

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self

In [13]:
np.random.seed(123)
custom_tree = CustomDecisionTree()
grid_search = GridSearchCV(
    estimator=custom_tree,
    param_grid=tree_grid, 
    cv=10, 
    n_jobs=7, 
    verbose=0, 
    scoring='f1_macro',
)

with parallel_backend('threading'):
    grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

joblib.dump(best_params, 'tree_best_params.joblib')
print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'ccp_alpha': 0.0, 'criterion': 'gini', 'handling_unbalance_data': 'imbalanced', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best cross-validation score: 0.9717359028293167


In [32]:
best_params = joblib.load('tree_best_params.joblib')
clf_tree = CustomDecisionTree(**best_params)
print('CV results:')
np.random.seed(123)
grid_search = GridSearchCV(
    estimator=clf_tree,
    param_grid=dict(zip(best_params.keys(), [[x] for x in best_params.values()])),
    cv=10, 
    n_jobs=-1, 
    verbose=0, 
    scoring='f1_macro',
)
with parallel_backend('threading'):
    grid_search.fit(X_train, y_train)
print('Mean macro f1-score', grid_search.cv_results_['mean_test_score'][0])
print('Standard deviation of the cv macro F1-scores', grid_search.cv_results_['std_test_score'][0])
print('\nResults on Test Data')
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(f'F1 score {f1_score(y_test, y_pred_tree, average='macro')}')
print(f'accuracy {metrics.accuracy_score(y_test, y_pred_tree)}')
print(metrics.classification_report(y_test, y_pred_tree))

CV results:
Mean macro f1-score 0.9690551711883112
Standard deviation of the cv macro F1-scores 0.006732256098763957

Results on Test Data
F1 score 0.9689756140607119
accuracy 0.9753158977372907
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       898
           1       0.97      0.98      0.98       662
           2       1.00      1.00      1.00       535
           3       1.00      1.00      1.00       459
           4       0.92      0.92      0.92       388
           5       0.89      0.89      0.89       333
           6       0.99      1.00      1.00       128

    accuracy                           0.98      3403
   macro avg       0.97      0.97      0.97      3403
weighted avg       0.98      0.98      0.98      3403



### KNN

In [25]:
X = np.float_(df.drop('Class', axis=1).values)
y = np.array([classes.index(x) for x in df['Class']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[:,:10])
X_test_num = scaler.transform(X_test[:,:10])
X_train = np.hstack([X_train_num, X_train[:,10:]])
X_test = np.hstack([X_test_num, X_test[:, 10:]])

In [26]:
X_test[0]

array([-0.51096551, -1.59675482,  1.03777237,  1.76131561, -0.22412244,
        1.17675536,  0.81102102,  0.97680419,  0.49076574, -0.63987   ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ])

In [27]:
# in each of the numerical distances the distances are sorted with these metrics, additionally the votes are also weighted with these metric
numerical_distance_metrics = ['minkowski', 'inverse_cosine_similarity',
                              'mahalonobis_distance']
numerical_distance_parameters = {
    'minkowski': {'p': [1, 2, 3]},
    'inverse_cosine_similarity': None,
    'mahalonobis_distance': None
}

categorical_distance_measures = ['hamming', 'jacard']
handling_imbalance = ['smote', 'nothing', 'under']

k_range = np.arange(1, 30, 2)
numerical_categorical_distance_weight_range = np.arange(0.1, 1, 0.1)

In [28]:
def minkowski_distance(X, Y, p):
    return cdist(X, Y, metric='minkowski', p=p)

def inverse_cosine_similarity(X, Y):
    return cdist(X, Y, metric='cosine')

def mahalanobis_distance(X, Y):
    VI = np.linalg.inv(np.cov(X.T))
    return cdist(X, Y, metric='mahalanobis', VI=VI)

def inverse_gaussian_kernel(X, Y, sigma):
    sq_dists = cdist(X, Y, metric='sqeuclidean')
    return 1 / (np.exp(-sq_dists / (2 * sigma**2)) + 1e-10)

def hamming_distance(X, Y):
    return cdist(X, Y, metric='hamming')

def jaccard_distance(X, Y):
    return cdist(X, Y, metric='jaccard')


In [29]:
class CustomKNN:
    def __init__(
            self, 
            k=5, 
            numerical_metric='minkowski', 
            numerical_params=None, 
            categorical_metric='hamming', 
            num_cat_weight=0.5, 
            imbalance_handling='nothing',
            num_numerical=10):
        self.k = k
        self.numerical_metric = numerical_metric
        self.numerical_params = numerical_params
        self.categorical_metric = categorical_metric
        self.num_cat_weight = num_cat_weight
        self.imbalance_handling = imbalance_handling
        self.num_num = num_numerical

    def fit(self, X, y):
        X_num = X[:, :self.num_num]
        X_cat = X[:, self.num_num:]
        if self.imbalance_handling == 'smote':
            sm = SMOTE(random_state=42)
            X_combined, y_combined = sm.fit_resample(np.hstack((X_num, X_cat)), y)
            X_num, X_cat = X_combined[:, :X_num.shape[1]], X_combined[:, X_num.shape[1]:]
            self.y_train = y_combined
        elif self.imbalance_handling == 'under':
            rus = RandomUnderSampler(random_state=42)
            X_combined, y_combined =  rus.fit_resample(np.hstack((X_num, X_cat)), y)
            X_num, X_cat = X_combined[:, :X_num.shape[1]], X_combined[:, X_num.shape[1]:]
            self.y_train = y_combined
        else:
            self.y_train = y
        self.X_num_train = X_num
        self.X_cat_train = X_cat
        return self

    def predict(self, X):
        X_num = X[:, :self.num_num]
        X_cat = X[:, self.num_num:]
        num_distances = self._compute_numerical_distances(X_num)
        cat_distances = self._compute_categorical_distances(X_cat)
        
        combined_distances = self.num_cat_weight * num_distances + \
                             (1 - self.num_cat_weight) * cat_distances
        
        return self._get_predictions(combined_distances)

    def _compute_numerical_distances(self, X_num):
        if self.numerical_metric == 'minkowski':
            return minkowski_distance(X_num, self.X_num_train, p=self.numerical_params['p'])
        elif self.numerical_metric == 'inverse_cosine_similarity':
            return inverse_cosine_similarity(X_num, self.X_num_train)
        elif self.numerical_metric == 'mahalanobis_distance':
            return mahalanobis_distance(X_num, self.X_num_train)
        elif self.numerical_metric == 'inverse_gaussian_kernel':
            return inverse_gaussian_kernel(X_num, self.X_num_train, sigma=self.numerical_params['sigma'])

    def _compute_categorical_distances(self, X_cat):
        if self.categorical_metric == 'hamming':
            return hamming_distance(X_cat, self.X_cat_train)
        elif self.categorical_metric == 'jacard':
            return jaccard_distance(X_cat, self.X_cat_train)
        
    def get_params(self, deep=True):
        return {"k": self.k, "numerical_metric": self.numerical_metric,
                "numerical_params": self.numerical_params,
                "categorical_metric": self.categorical_metric,
                "num_cat_weight": self.num_cat_weight,
                "imbalance_handling": self.imbalance_handling}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def _get_predictions(self, distances):
        neighbors_idx = np.argpartition(distances, self.k, axis=1)[:, :self.k]
        neighbors_labels = self.y_train[neighbors_idx]
        mode_data = mode(neighbors_labels, axis=1)
        return mode_data.mode.ravel()


Grid Search over minkowski distances

In [24]:
param_grid = {
    'k': range(1, 30, 2),
    'numerical_metric': ['inverse_cosine_similarity'],  # Adjust according to your previous setup
    'numerical_params': [None],
    'categorical_metric': ['hamming', 'jacard'],
    'num_cat_weight': np.arange(0.1, 1, 0.1),
    'imbalance_handling': ['nothing', 'smote', 'under']
}
grid_search = GridSearchCV(CustomKNN(), param_grid, n_jobs=7, cv=10, scoring=make_scorer(f1_score, average='macro'))

with parallel_backend('threading'): 
    grid_search.fit(X_train, y_train) 

joblib.dump(grid_search.best_params_, 'inverse_cosine_params.joblib')
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'categorical_metric': 'hamming', 'imbalance_handling': 'nothing', 'k': 9, 'num_cat_weight': 0.1, 'numerical_metric': 'inverse_cosine_similarity', 'numerical_params': None}
Best cross-validation score: 0.9686182067910734


In [25]:
param_grid = {
    'k': range(1, 30, 2),
    'numerical_metric': ['minkowski'], 
    'numerical_params': [{'p': p} for p in range(1, 4)],
    'categorical_metric': ['hamming', 'jacard'],
    'num_cat_weight': np.arange(0.1, 1, 0.1),
    'imbalance_handling': ['nothing', 'smote', 'under']
}
grid_search = GridSearchCV(CustomKNN(), param_grid, n_jobs=-1, cv=10, scoring=make_scorer(f1_score, average='macro'))

with parallel_backend('threading'):
    grid_search.fit(X_train, y_train) 

joblib.dump(grid_search.best_params_, 'minkowski_params.joblib')
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'categorical_metric': 'hamming', 'imbalance_handling': 'smote', 'k': 23, 'num_cat_weight': 0.1, 'numerical_metric': 'minkowski', 'numerical_params': {'p': 1}}
Best cross-validation score: 0.9752985875304839


In [26]:
param_grid = {
    'k': range(1, 30, 2),
    'numerical_metric': ['mahalanobis_distance'],  # Adjust according to your previous setup
    'numerical_params': [None],
    'categorical_metric': ['hamming', 'jacard'],
    'num_cat_weight': np.arange(0.1, 1, 0.1),
    'imbalance_handling': ['nothing', 'smote', 'under']
}
grid_search = GridSearchCV(CustomKNN(), param_grid, n_jobs=-1, cv=10, scoring=make_scorer(f1_score, average='macro'))

with parallel_backend('threading'):
    grid_search.fit(X_train, y_train) 

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Best parameters: {'categorical_metric': 'hamming', 'imbalance_handling': 'smote', 'k': 23, 'num_cat_weight': 0.1, 'numerical_metric': 'mahalanobis_distance', 'numerical_params': None}
Best cross-validation score: 0.9721560607831489


In [27]:
joblib.dump(grid_search.best_params_, 'mahalanobis_params.joblib')

['mahalanobis_params.joblib']

In [28]:
params = joblib.load('minkowski_params.joblib')
knn = CustomKNN(**params)
print('Performance of the cross validation')

np.random.seed(123)
grid_search = GridSearchCV(
    estimator=CustomKNN(),
    param_grid=dict(zip(params.keys(), [[x] for x in params.values()])),
    cv=10, 
    n_jobs=3, 
    verbose=0, 
    scoring='f1_macro',
)
with parallel_backend('threading'):
    grid_search.fit(X_train, y_train)
print(grid_search.cv_results_['mean_test_score'][0])
print(grid_search.cv_results_['std_test_score'][0])

print('\nPerformance of the test scores')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(f"F1_score: {f1_score(y_test, y_pred, average='macro')}")
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

Performance of the cross validation
0.9752985875304839
0.006676352065562572

Performance of the test scores
F1_score: 0.9733258983700012
Accuracy: 0.9779606229797237


In [43]:
# Feture selection: Backward stagewise selection
numerical_features = list(np.arange(0, 10, 1))
categorical_features = list(np.arange(0, X_train.shape[1] - 10, 1))
best_score= 0.9752985875304839
while True:
    best_j = -1
    improved = False
    for j in range(len(numerical_features) + len(categorical_features)):
        if (len(numerical_features)==1 and j == 0) or \
            (len(categorical_features)==1 and j == len(numerical_features) + len(categorical_features) - 1):
            continue
        temp_numerical_features = numerical_features.copy()
        temp_categorical_features  = categorical_features.copy()
        if j < len(temp_numerical_features):
            temp_numerical_features.pop(j)
        else:
            temp_categorical_features.pop(j-len(temp_numerical_features))
        new_X_train_num = X_train_num[:, temp_numerical_features]
        new_X_train_cat = X_train[:, 10:][:, temp_categorical_features]
        grid_search = GridSearchCV(
            estimator=CustomKNN(num_numerical=len(temp_numerical_features)),
            param_grid=dict(zip(params.keys(), [[x] for x in params.values()])),
            cv=10, 
            n_jobs=3, 
            verbose=0, 
            scoring='f1_macro',
        )
        with parallel_backend('threading'):
            grid_search.fit(np.hstack((new_X_train_num, new_X_train_cat)), y_train)
        current_score = grid_search.cv_results_['mean_test_score'][0]
        if current_score > best_score:
            best_j = j
            best_score = current_score
            improved = True
    if not improved:
        break
    if best_j < len(numerical_features):
        numerical_features.pop(best_j)
        print('numerical', best_j)
    else:
        categorical_features.pop(best_j-len(numerical_features))
        print('categorical', best_j)
print(numerical_features, categorical_features)

numerical 9
numerical 7
numerical 5
categorical 7
[0, 1, 2, 3, 4, 6, 8] [1, 2, 3, 4]


In [ ]:
joblib.dump(numerical_features, 'backward_selection_numerical.joblib')
joblib.dump(categorical_features, 'backward_selection_categorical.joblib')

In [30]:
numerical_features = joblib.load('backward_selection_numerical.joblib')
categorical_features = joblib.load('backward_selection_categorical.joblib')

In [33]:
params = joblib.load('minkowski_params.joblib')
knn = CustomKNN(**params)
print('Performance of the cross validation')

new_X_train_num = X_train_num[:, numerical_features]
new_X_train_cat = X_train[:, 10:][:, categorical_features]
np.random.seed(123)
grid_search = GridSearchCV(
    estimator=CustomKNN(num_numerical=len(numerical_features)),
    param_grid=dict(zip(params.keys(), [[x] for x in params.values()])),
    cv=10, 
    n_jobs=3, 
    verbose=0, 
    scoring='f1_macro',
)
with parallel_backend('threading'):
    grid_search.fit(np.hstack((new_X_train_num, new_X_train_cat)), y_train)
print(grid_search.cv_results_['mean_test_score'][0])
print(grid_search.cv_results_['std_test_score'][0])

print('\nPerformance of the test scores')
knn.fit(np.hstack((new_X_train_num, new_X_train_cat)), y_train)
X_test_new = np.hstack(( X_test_num[:, numerical_features], X_test[:, 10:][:, categorical_features]))
y_pred_knn = knn.predict(X_test_new)
print(f"F1_score: {f1_score(y_test, y_pred_knn, average='macro')}")
print('Accuracy:', metrics.accuracy_score(y_test, y_pred_knn))
print(metrics.classification_report(y_test, y_pred_knn))

Performance of the cross validation
0.9805147387374541
0.005954950017601164

Performance of the test scores
F1_score: 0.9799767887963353
Accuracy: 0.9841316485454011
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       898
           1       0.98      0.99      0.98       662
           2       1.00      1.00      1.00       535
           3       1.00      1.00      1.00       459
           4       0.94      0.95      0.95       388
           5       0.95      0.92      0.93       333
           6       0.99      1.00      1.00       128

    accuracy                           0.98      3403
   macro avg       0.98      0.98      0.98      3403
weighted avg       0.98      0.98      0.98      3403



In [50]:
print(f'All the numerical features: {list(df.columns)[:10]}')
print(f'Chosen numerical features: {list(df.columns[numerical_features])}')

All the numerical features: ['Area', 'AspectRation', 'Solidity', 'roundness', 'ShapeFactor1', 'ShapeFactor4', 'ShapeFactor5', 'Sort order', 'Extent', 'ShapeFactor6']
Chosen numerical features: ['Area', 'AspectRation', 'Solidity', 'roundness', 'ShapeFactor1', 'ShapeFactor5', 'Extent']


In [52]:
print(f'All the categorical features: {list(df.columns)[10:-1]}')
print(f'Chosen categorical features: {list(df.columns[10:][categorical_features])}')

All the categorical features: ['Constantness', 'Colour_black', 'Colour_brown', 'Colour_green', 'Colour_white']
Chosen categorical features: ['Colour_black', 'Colour_brown', 'Colour_green', 'Colour_white']


## Bowker's Test

In [38]:
num_classes = 7

continguency_table = np.zeros((num_classes, num_classes))
for pred1, pred2 in zip(y_pred_tree, y_pred_knn):
    continguency_table[pred1, pred2] += 1

n = len(y_pred_knn)
test_statistic = 0
for i in range(num_classes-1):
    for j in range(i+1, num_classes):
        if (continguency_table[i, j] + continguency_table[j, i]) != 0:
            test_statistic += (continguency_table[i, j] - continguency_table[j, i])**2 / (continguency_table[i, j] + continguency_table[j, i])
        else:
            test_statistic += 0

degrees_freedom = 0.5 * num_classes * (num_classes - 1)
p_value = 1 - chi2.cdf(test_statistic, degrees_freedom)

print(f'The Chi-square statistic {test_statistic} has p-value {p_value}')

The Chi-square statistic 5.34965034965035 has p-value 0.9997701263630812


In [35]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6])

In [40]:
metrics.confusion_matrix(y_pred_tree, y_pred_knn)

array([[898,   0,   0,   0,   0,   0,   0],
       [  0, 652,   0,   0,   6,   6,   0],
       [  0,   0, 534,   0,   0,   0,   0],
       [  0,   0,   0, 459,   0,   0,   0],
       [  0,   7,   0,   0, 361,  17,   1],
       [  0,  10,   0,   0,  27, 296,   0],
       [  0,   0,   0,   0,   0,   1, 128]], dtype=int64)